In [1]:
import requests
from bs4 import BeautifulSoup as bs
# import pandas as pd
import json
import tqdm
from os import mkdir
from os.path import join
from shutil import rmtree

In [84]:
headers = {
    'Accept-Encoding': 'gzip, deflate, sdch',
    'Accept-Language': 'en-US,en;q=0.8',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
}
META_PREFICS = "https://www.metacritic.com"
USER_POSTFICS = "/user-reviews"
CRITIC_POSTFICS = "/critic-reviews"

save_path ="/Users/fedor/Desktop/meta_project/data/metacritic/critic_review/"
strange_game_list = []

In [105]:

def parse_critic_page(URL:str,
                       local_save_path:str,
                       json_save:bool = False):

    r = requests.get(URL, headers=headers)
    soup = bs(r.text, "html.parser")

    if soup.find(class_="body product_reviews") is not None:
        base = soup.find(class_="body product_reviews")
    else:
        strange_game_list.append(URL)
        return

    titels = base.find_all(class_="review_content")
    rewiev_count = 0
    for t in titels:

        name = t.find(class_="source").text
        ref =  t.find(class_="external")

        if ref is not None:
            ref = ref['href']
        else:
            ref = None

        score = t.find(class_="review_grade").div.text
        date = t.find(class_="date")

        if date is not None:
            date = date.text
        else:
            date = None

        text = t.find(class_="review_body").text

        if json_save:
            with open( local_save_path +f"/review_{rewiev_count}.json", 'w') as f:
                json.dump({ "name": name, "ref": ref, "score": score,
                            "date":date, "text":text}, f)
        else:
            return { "name": name, "ref": ref, "score": score, "date":date, "text":text}
        rewiev_count +=1


In [86]:
games_name_changed = []
with open('/Users/fedor/Desktop/meta_project/data/metacritic/url/critic_refs/final_critic_refs.json') as f:
    critic_ref = json.load(f)

In [106]:
strange_game_list = []

In [132]:
actual_start_pos = 5005+1129+4095+746

In [134]:
loc_path = save_path
for game_name in tqdm.tqdm(list(critic_ref.keys())[actual_start_pos : ]):
    safe_game_name = game_name.replace("/","_")

    if safe_game_name != game_name:
        games_name_changed.append(game_name)

    try:
        mkdir( loc_path + safe_game_name )
    except FileExistsError:
        rmtree(join(loc_path + safe_game_name))
        mkdir(loc_path + safe_game_name)


    for platform in critic_ref[game_name]:
        if platform == "iPhone/iPad":
            safe_platform = "iPhone_iPad"
        else:
            safe_platform = platform

        mkdir(loc_path+safe_game_name+"/"+safe_platform)

        game_link = critic_ref[game_name][platform]
        parse_critic_page(game_link
                          ,loc_path + safe_game_name + "/" + safe_platform
                          , True)



100%|██████████| 1952/1952 [59:30<00:00,  1.83s/it] 


## Аналитика полученных результатов

In [ ]:
critic_names = {}

получение списка отзывов на игрв от популярных издательств

In [223]:
from pathlib import Path
names_game_dict = {}

# assign directory
directory = "/Users/fedor/Desktop/meta_project/data/metacritic/critic_review"

# iterate over files in
# that directory
names = Path(directory).glob('*')
for n in tqdm.tqdm(names):
    rev = Path(n).glob('*')
    for r in rev:
        r = Path(r).glob('*')
        for _ in r:
            with open(_) as f:
                templates = json.load(f)
                name = templates["name"]
                if name in critic_names_list[:35]:
                    if name in names_game_dict.keys():
                        names_game_dict[name].append(str(n).split("/")[-1])
                    else:
                        names_game_dict[name] = [str(n).split("/")[-1]]



12899it [03:27, 62.09it/s]


ранжирование игр по популярности

In [181]:
critic_names_list = list(critic_names.keys())

In [186]:
critic_names_list.sort(key = lambda x:critic_names[x], reverse=True)

In [241]:
for c in  critic_names_list[:35]:
    print(c, critic_names[c])


IGN 12106
GameSpot 10849
Game Informer 6817
Multiplayer.it 5622
Worth Playing 5617
Gaming Age 5599
Cheat Code Central 5579
Vandal 5447
GameZone 5274
Game Revolution 4854
Everyeye.it 4643
SpazioGames 4299
Eurogamer 4210
GameSpy 4150
ZTGD 4145
Eurogamer Italy 4097
Destructoid 4089
GamingTrend 4056
3DJuegos 3904
GamePro 3788
COGconnected 3706
Metro GameCentral 3593
Games Master UK 3550
GamesRadar+ 3543
Hardcore Gamer 3462
4Players.de 3432
God is a Geek 3426
Gaming Nexus 3418
DarkStation 3393
Meristation 3312
GameCritics 3300
1UP 3286
Edge Magazine 3250
NintendoWorldReport 3233
PC Gamer 3232


In [227]:
for c in  critic_names_list[:20]:
    print(c, critic_names[c])

IGN 12106
GameSpot 10849
Game Informer 6817
Multiplayer.it 5622
Worth Playing 5617
Gaming Age 5599
Cheat Code Central 5579
Vandal 5447
GameZone 5274
Game Revolution 4854
Everyeye.it 4643
SpazioGames 4299
Eurogamer 4210
GameSpy 4150
ZTGD 4145
Eurogamer Italy 4097
Destructoid 4089
GamingTrend 4056
3DJuegos 3904
GamePro 3788


покрытие раличных тайтлов игр при рассмотрении критики различного количества игровых издательств

In [238]:
total_games_cover = set()
for i in range(5,50,5):
    for n in names_game_dict:
        if n in critic_names_list[:i]:
            for g in names_game_dict[n]:
                total_games_cover.add(g)
    print(i,"-",len(total_games_cover))

5 - 11243
10 - 11962
15 - 12317
20 - 12535
25 - 12678
30 - 12783
35 - 12827
40 - 12827
45 - 12827


общее количество отзывов при рассмотрении критики различного количества игровых издательств

In [239]:
hist = []
sum_hist = []
for i in range(5,50,5):
    sum = 0
    for c in  critic_names_list[:i]:
        sum += critic_names[c]
    print(i, '-',sum)

5 - 41011
10 - 67764
15 - 89211
20 - 109145
25 - 126999
30 - 143980
35 - 160281
40 - 175736
45 - 190227


In [240]:
critic_names_list[:35]

['IGN',
 'GameSpot',
 'Game Informer',
 'Multiplayer.it',
 'Worth Playing',
 'Gaming Age',
 'Cheat Code Central',
 'Vandal',
 'GameZone',
 'Game Revolution',
 'Everyeye.it',
 'SpazioGames',
 'Eurogamer',
 'GameSpy',
 'ZTGD',
 'Eurogamer Italy',
 'Destructoid',
 'GamingTrend',
 '3DJuegos',
 'GamePro',
 'COGconnected',
 'Metro GameCentral',
 'Games Master UK',
 'GamesRadar+',
 'Hardcore Gamer',
 '4Players.de',
 'God is a Geek',
 'Gaming Nexus',
 'DarkStation',
 'Meristation',
 'GameCritics',
 '1UP',
 'Edge Magazine',
 'NintendoWorldReport',
 'PC Gamer']